# Sherwood Station Power Company BIORAM Example

This notebook demonstrates the BIORAM compliance example for an imagined company using the BOOST Python reference implementation with proper schema validation and JSON-LD compatibility. 

## Key Features

- **Schema Compliance**: All entities conform to BOOST core schemas
- **Dynamic Validation**: Uses schema-driven validation from the Python reference implementation
- **JSON-LD Support**: Proper semantic web compatibility with `@context`, `@type`, `@id`
- **Automatic Enum Validation**: Uses current schema enums for field validation
- **ID Pattern Compliance**: Follows BOOST ID conventions (`ORG-*`, `TXN-*`, etc.)
- **Future-Proof Design**: Adapts automatically to schema changes

## Set up and import BOOST Client


In [ ]:
import sys
import os
import json
from datetime import datetime, date
from pathlib import Path

# Add BOOST Python reference implementation to path
boost_python_path = Path("../../../../drafts/current/reference-implementations/python")
sys.path.insert(0, str(boost_python_path.resolve()))

# Import BOOST client
from boost_client import create_client

print("✓ BOOST Python reference implementation imported successfully")

In [ ]:
# Initialize BOOST client with schema path
schema_path = str(Path("../../../../drafts/current/schema").resolve())
client = create_client(schema_path=schema_path)

# Get schema information
schema_info = client.get_schema_info()
print(f"✓ BOOST client initialized")
print(f"Available entities: {schema_info['available_entities']}")
print(f"Schema loader: {schema_info['schema_loader']}")
print(f"Validator: {schema_info['validator']}")

## Dynamic Schema Introspection

Let's explore what enum values are available from the current schemas. 


In [ ]:
# Get available enum values from schemas
org_types = client.get_available_enum_values('organization', 'organizationType')
regulated_entity_types = client.get_available_enum_values('organization', 'regulatedEntityType')
fuel_categories = client.get_available_enum_values('transaction', 'fuelCategory')
transaction_statuses = client.get_available_enum_values('transaction', 'transactionStatus')

print("Available enum values from current schemas:")
print(f"Organization types: {org_types}")
print(f"Regulated entity types: {regulated_entity_types}")
print(f"Fuel categories: {fuel_categories}")
print(f"Transaction statuses: {transaction_statuses}")

## Create BOOST-compliant Organization 

Let's create the Sherwood Station Power Company organization using proper BOOST schema validation. 

In [ ]:
org_id = client.generate_id("organization", "SHERWOOD-STATION")
print(f"Generated organization ID: {org_id}")

sherwood_station = client.create_organization(
    organization_id=org_id,
    organizationName="Sherwood Station Power Company",
    organizationType="producer",
    contactEmail="contact@sherwoodstation.com",
    contactPhone="+1-555-123-4567",
    regulatedEntityType="producer",
    operationalStatus="active",
    facilityCapacity={
        "annualCapacity": 500000,
        "unit": "MJ"
    }
)


## Create Upstream Organizations

Since Sherwood is an end consumer of mill waste, let's create the upstream suppliers that provide Sherwood with feedstock. 


In [ ]:
suppliers = []
supplier_data = [
    {
        "name": "Anderson Valley Lumber Company",
        "suffix": "AVL-001"
    },
    {
        "name": "Anderson Valley Lumber Company",
        "suffix": "AVL-001"
    },
]  
for supplier_info in supplier_data:
    supplier_id = client.generate_id("organization", supplier_info["suffix"])
    supplier = client.create_organization(
        organization_id=supplier_id,
        name=supplier_info["name"],
        org_type="producer",
        contactEmail="contact@andersonvalleylumber.com",
        contactPhone="+1-555-123-4567",
        regulatedEntityType="producer",
        operationalStatus="active",
    )
    suppliers.append(supplier)
    print(f"Created supplier: {supplier_info['name']} (ID: {supplier_id})")



## Create BOOST-compliant Transactions

Now, let's create transactions using the proper BOOST transaction schema with all required fields. 



In [ ]:
original_transactions = [
    {
        "transactionDate": "2025-01-15",
        "contractValue": 875000.0,
        "contractCurrency": "USD",
        "transactionStatus": "completed",
        "feedstockType": "mill_waste",
        "feedstockVolume": 4375,
        "feedstockUnit": "tons",
        "supplierId": "SUPP-MILL-001",
        "OrganizationId": "ORG-BIORAM-001",
        "deliveryLocation": {"city": "Oakland", "state": "CA"}
    },
    {
        "transactionDate": "2025-01-28",
        "contractValue": 1250000.0,
        "contractCurrency": "USD",
        "transactionStatus": "completed",
        "feedstockType": "mill_waste",
        "feedstockVolume": 6250,
        "feedstockUnit": "tons",
        "supplierId": "SUPP-MILL-001",
        "OrganizationId": "ORG-BIORAM-001",
        "deliveryLocation": {"city": "Fresno", "state": "CA"}
    },
    {
        "transactionDate": "2025-02-12",
        "contractValue": 650000.0,
        "contractCurrency": "USD",
        "transactionStatus": "completed",
        "feedstockType": "mill_waste",
        "feedstockVolume": 3250,
        "feedstockUnit": "tons",
        "supplierId": "SUPP-MILL-002",
        "OrganizationId": "ORG-BIORAM-002",
        "deliveryLocation": {"city": "San Jose", "state": "CA"}
    },
    {
        "transactionDate": "2025-02-25",
        "contractValue": 425000.0,
        "contractCurrency": "USD",
        "transactionStatus": "completed",
        "feedstockType": "mill_waste",
        "feedstockVolume": 2125,
        "feedstockUnit": "tons",
        "supplierId": "SUPP-MILL-002",
        "OrganizationId": "ORG-BIORAM-002",
        "deliveryLocation": {"city": "Sacramento", "state": "CA"}
    },
    {
        "transactionDate": "2025-03-10",
        "contractValue": 1100000.0,
        "contractCurrency": "USD",
        "transactionStatus": "completed",
        "feedstockType": "mill_waste",
        "feedstockVolume": 5500,
        "feedstockUnit": "tons",
        "supplierId": "SUPP-MILL-003",
        "OrganizationId": "ORG-BIORAM-003",
        "deliveryLocation": {"city": "Bakersfield", "state": "CA"}
    },
    {
        "transactionDate": "2025-03-22",
        "contractValue": 775000.0,
        "contractCurrency": "USD",
        "transactionStatus": "completed",
        "feedstockType": "mill_waste",
        "feedstockVolume": 3875,
        "feedstockUnit": "tons",
        "supplierId": "SUPP-MILL-003",
        "OrganizationId": "ORG-BIORAM-003",
        "deliveryLocation": {"city": "Los Angeles", "state": "CA"}
    }
]

print(f"Converting {len(original_transactions)} BIORAM transactions to BOOST format...")

In [ ]:
boost_transactions = []

for i, txn_data in enumerate(original_transactions, 1):
    # Generate proper BOOST transaction ID
    txn_id = client.generate_id("transaction", f"2025-Q1-{i:03d}")
    
    # Get customer organization data
    supplier = suppliers[txn_data["supplierIndex"]]
    supplier_data = supplier.model_dump(by_alias=True)
    
    # Generate proper Supplier ID referencing the organization
    supplier_id = client.generate_id("transaction", f"SUPP-{supplier_data['organizationId'].replace('ORG-', '')}")
    
    # Calculate contract value
    contract_value = txn_data["feedstockVolume"] * txn_data["feedstockPrice"]
    transaction = client.create_transaction(
        transaction_id=txn_id,
        organization_id=supplier_data["organizationId"],
        customer_id=supplier_data["organizationId"],
        transaction_date=txn_data["transactionDate"],
        contract_value=contract_value,
        contract_currency="USD",
        transaction_status="completed",
        feedstock_type=txn_data["feedstockType"],
        # bioram Specific fields
        volumeDelivered = txn_data["feedstockVolume"], # TODO come back to this one 
        feedstockUnit = "bone-dry_tonnes",
        fuelType = feedstock_type, # or should it be morte specific? 
        reportingPeriod = "2025-Q1",
        haulDistance = txn_data["haulDistance"],
        haulUnit = "mile",
        

        